In [1]:
import os.path
import openpyxl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt as sqrt

In [2]:
#
diametre = np.array([15, 20, 17.5, 23])
#vom folosi 15 diametru standard pentru biesele la care analizam pierderea locala de sarcina
#vom folosi restul diametrelor pentru masuratorile pierderii liniare
#am creat o lista de labeluri pe care le vom folosi la grafice
labels = np.array([f'{i} mm' for i in diametre])

#datele experimentale pierdere liniara de sarcina

debite_pierdere_liniara ={'1' : np.array([0.880, 0.844, 0.742, 0.608, 0.471, 0.403, 0.323, 0.293, 0.107]),
                          '2' : np.array([1.196, 0.996, 0.882, 0.805, 0.701, 0.640, 0.547, 0.405, 0.330]),
                          '3' : np.array([1.075, 0.917, 0.718, 0.692, 0.613, 0.516, 0.390, 0.238, 0.233]), 
                          '4' : np.array([1.24, 1.05, 0.88, 0.80, 0.79, 0.69, 0.60, 0.5, 0.4])
                         }
presiuni_pierdere_liniara = {'1' : np.array([148.4, 135.13, 107.67, 75.38, 47.53, 35.64, 23.60, 20.07, 2.58]),
                             '2' : np.array([43.32, 29.58, 24.75, 20.46, 15.58, 13.32, 9.79, 5.42, 3.58]),
                             '3' : np.array([89.95, 66.33, 49.07, 39.17, 31.04, 22.68, 13.25, 4.81, 4.5]),
                             '4' : np.array([35.3, 25.3, 18.5, 14.8, 15.1, 11.5, 8.8, 5.81, 3.30])
                            }

#datele experimentale pierdere locala de saricna

debite_pierdere_locala = {'curba 90': np.array([0.509, 0.446, 0.398, 0.345, 0.302, 0.238, 0.199]),
                          'curba 45': np.array([0.60, 0.542, 0.509, 0.449, 0.407, 0.320, 0.232]),
                          'cot 135': np.array([0.511, 0.438, 0.390, 0.362, 0.310, 0.261]),
                          'cot 90':  np.array([0.603, 0.549, 0.500, 0.440, 0.391, 0.334, 0.300, 0.136]),
                          'robinet cu sertar': np.array([0.55, 0.50, 0.45, 0.40, 0.35, 0.30, 0.25, 0.15]),
                          'robinet cu ventil': np.array([0.55, 0.51, 0.448, 0.402, 0.35, 0.259, 0.158]),
                          '2 robineti cu ventil': np.array([0.55, 0.507, 0.455, 0.407, 0.348, 0.312, 0.254, 0.14])
                         }

presiuni_pierdere_locala = {'curba 90': np.array([1653, 1324.24, 1036.97, 763.29, 559.17, 311.50, 158.3]),
                            'curba 45': np.array([22.91, 18.77, 17.08, 13.71, 10.64, 7.34, 3.50]),
                            'cot 135': np.array([273.13, 200.49, 160.52, 139.12, 101, 71.47]),
                            'cot 90': np.array([204.86, 168.73, 140.89, 109.90, 85.89, 65.56, 49.37, 7.88]),
                            'robinet cu sertar': np.array([1119, 885.5, 715.34, 585.09, 437.5, 326.82, 230, 71.3]),
                            'robinet cu ventil': np.array([406.44, 336.56, 260.09, 208.31, 161.88, 85.12, 30.35]),
                            '2 robineti cu ventil': np.array([286.86, 238, 191.3, 155.54, 114.04, 88.96, 58.73, 16.47])
                           }

Zeta are formula: $\zeta=\frac{\Delta p D^4}{0.0826 \rho g L Q^2}$ si reprezinta coeficientul de pierdere locala de sarcina

Lambda are formula: $\lambda=\frac{\Delta p D^5}{0.0826 \rho g L Q^2}$ si reprezinta coeficientul de pierdere liniara de sarcina

Folosim pentru Criteriul Reynolds formula: $Re = \frac{4Q}{\nu \pi D}$

Folosim pentru pierderea de sarcina formula $ h = \frac{\Delta p}{\rho g}$

In [3]:
class My_class:
    def __init__(self, presiuni, debite, diametru, key = None):
        self.presiuni = presiuni * pow(10, 2) #conversie mbar --> Pa
        self.debite    = debite    * pow(10,-3) #conversie  dm^3 --> m^3
        self.diametru = diametru * pow(10,-3) #conversie mm -> m
        self.key = key
    def __repr__(self):
        return f'Aceasta teava are grosimea de :{self.diametru}'
    
    def lamda(self, ro = 1000, g = 9.81, L = 1):
        return self.presiuni*pow(self.diametru, 5) / ( 0.0826*ro*g*L*pow(self.debite,2) )
    def zeta(self, ro = 1000, g = 9.81):
        return self.presiuni*pow(self.diametru, 4) / ( 0.0826*ro*g*pow(self.debite,2) )
    
    def reynolds(self, niu = pow(10, -6) ):
        return 4*self.debite / (np.pi*niu*self.diametru)    
    def pierdere_sarcina(self, ro = 1000, g = 10):
        return self.presiuni / (ro*g)
    def viteza_medie(self):
        return self.debite/ pow(self.diametru, 2)

    @property
    def df_liniar (self):
        data ={'Q (m^3/s)': self.debite,
               '\u0394p (Pa)': self.presiuni,
               'D(m)': self.diametru,
               '\u03BB': self.lamda(), #lamda
               'Re': self.reynolds(), 
               'S': self.key
              }
        df = pd.DataFrame(data)
        return df

    @property
    def df_local (self):
        data ={'Q (m^3/s)': self.debite,
               '\u0394p (Pa)': self.presiuni,
               'D(m)': self.diametru,
               '\u03B6': self.zeta(), #zeta
               'Re': self.reynolds(),
               'Obs': self.key
              }
        df = pd.DataFrame(data)
        return df

In [4]:
def export_excel(date, pierdere):
    def control_flow (date, pierdere, writer):
        if pierdere == 'locala':
            for i in date:
                i.df_local.to_excel(writer, sheet_name= f'{i.key}')
        elif pierdere == 'liniara':
            for i in date:
                i.df_local.to_excel(writer, sheet_name= f'{i.diametru}')

    path = './Determinarea_exp.xlsx'
    check_file = os.path.isfile(path) 

    if check_file is False:
        with pd.ExcelWriter(path) as writer:
            control_flow(date, pierdere, writer)
    elif check_file is True:
        with pd.ExcelWriter(path, engine = 'openpyxl', mode = 'a', if_sheet_exists="overlay") as writer:
            control_flow(date, pierdere, writer)
    else:
        raise TypeError('pierderea poate fi doar locala sau liniara')

# Pierderi Liniare prelucrare

In [5]:
#s_1 = My_class(debit = debite_pierdere_liniara['1'], presiune = presiuni_pierdere_liniara['1'], diametru = diametre[0])
#print(s_1.df_lamda()) #testcase


masuratori_pierdere_liniara = []
i = 0
for key in debite_pierdere_liniara.keys():
    obj = My_class(debite = debite_pierdere_liniara[key], presiuni = presiuni_pierdere_liniara[key], diametru = diametre[i], key = key)
    masuratori_pierdere_liniara.append(obj)
    i += 1

In [6]:
#for i in masuratori_pierdere_liniara:
#    print(i.df_liniar)

## Grafic necorectat

In [ ]:
plt.title("Diagrama lui Gabi (Moody) pentru coeficientul de pierdere liniara de sarcina, fara corectarea erorilor")

for i in masuratori_pierdere_liniara:
    df = i.df_liniar
    plt.plot(df['Re'], df['\u03BB'], marker = 'x', label = f'{i.diametru*pow(10,3)}mm')
    
plt.legend()
plt.grid(True)
plt.ylabel('\u03BB', rotation = 0)
plt.xlabel('Reynolds')

plt.savefig("Diagrama lui Gabi (Moody) pentru coeficientul de pierdere liniara de sarcina, fara corectarea erorilor.png", dpi = 500)
plt.show()

## Grafic corectat

In [ ]:
#indexam de la 0 poz 3 = 2, poz 4 = 3
df_2 = masuratori_pierdere_liniara[2]
df_3 = masuratori_pierdere_liniara[3]

#print(df_2.df_liniar[['\u03BB','Re']])
#print(df_3.df_liniar[['\u03BB','Re']])

df_2 = df_2.df_liniar[['\u03BB','Re']]
df_3 = df_3.df_liniar[['\u03BB','Re']]

#print(df_2.loc[2])
#print(df_3.loc[3])

df_2.drop(axis = 1, index = 2, inplace = True)
df_3.drop(axis = 1, index = 3, inplace = True)

DF_liniare_c = [i.df_liniar for i in masuratori_pierdere_liniara]
DF_liniare_c[2] = df_2
DF_liniare_c[3] = df_3

#for i in masuratori_liniare_c:
#    print(i)

plt.title("Diagrama lui Gabi (Moody) pentru coeficientul de pierdere liniara de sarcina, cu corectarea erorilor")

k = 0
for i in DF_liniare_c:
    plt.plot(i['Re'], i['\u03BB'], marker = 'x', label = labels[k])
    k += 1
    
plt.legend()
plt.grid(True)
plt.ylabel('\u03BB', rotation = 0)
plt.xlabel('Reynolds')

plt.savefig("Diagrama lui Gabi (Moody) pentru coeficientul de pierdere liniara de sarcina, cu corectarea erorilor.png", dpi = 500)
plt.show()

## Diagrama pierderilor liniare de sarcina in functie de debite

In [ ]:
plt.title("Diagrama pierderilor liniare de sarcina in functie de debite")

for i in masuratori_pierdere_liniara:
    plt.plot(i.debite, i.pierdere_sarcina(), marker = 'o', label = i.diametru)
    
plt.legend()
plt.grid(True)
plt.ylabel('h [m]', rotation = 0)
#schimba label debit sa fie in litri
plt.xlabel('Q [m^3/s]')

plt.savefig("Diagrama pierderilor liniare de sarcina in functie de debite.png", dpi = 500)
plt.show()


## Diagrama pierderilor liniare de sarcina in functie de viteze

In [ ]:
plt.title("Diagrama pierderilor liniare de sarcina in functie de viteze")

for i in masuratori_pierdere_liniara:
    plt.plot(i.viteza_medie(), i.pierdere_sarcina(), marker = 'o', label = i.diametru)
    
plt.legend()
plt.grid(True)
plt.ylabel('h [m]', rotation = 0)
#schimba label debit sa fie in litri

plt.xlabel('v [m/s]')

plt.savefig("Diagrama pierderilor liniare de sarcina in functie de viteze", dpi = 500)
plt.show()

# Pierderi Locale Prelucrate

In [ ]:
masuratori_pierdere_locala_1 = []
#diametrul ramane constant la pierderea locala deoarece este acelasi pentru toate piesele
for key in debite_pierdere_locala.keys():
    if key == 'robinet cu sertar':
        break
    else:
        obj = My_class(debite = debite_pierdere_locala[key], presiuni = presiuni_pierdere_locala[key], diametru = diametre[0], key = key)
        masuratori_pierdere_locala_1.append(obj)

In [ ]:
masuratori_pierdere_locala_2 = []
debite_keys = list(debite_pierdere_locala.keys())
#diametrul ramane constant la pierderea locala deoarece este acelasi pentru toate piesele
for key in debite_keys[-3:]:
        obj = My_class(debite = debite_pierdere_locala[key], presiuni = presiuni_pierdere_locala[key], diametru = diametre[0], key = key)
        masuratori_pierdere_locala_2.append(obj)

#de_doua_ori_robinet_debite = debite_pierdere_locala['robinet cu ventil'] * 2
#de_doua_ori_robinet_presiuni = presiuni_pierdere_locala['robinet cu ventil'] * 2
#
#de_doua_ori_robinet = My_class(debite = de_doua_ori_robinet_debite, presiuni = de_doua_ori_robinet_presiuni, diametru = diametre[0], key = 'de doua ori robinet cu ventil')
#masuratori_pierdere_locala_2.append(de_doua_ori_robinet)

In [ ]:
#for i in masuratori_pierdere_locala_1:
#    print(i.df_local)

In [ ]:
#for i in masuratori_pierdere_locala_2:
#    print(i.df_local)

## Diagrama pierderilor locale de sarcina in functie de zeta si Re (1)

In [ ]:
plt.title("Diagrama pierderilor liniare de sarcina in functie de zeta (1)")

for i in masuratori_pierdere_locala_1:
    df = i.df_local
    plt.plot(df['Re'], df['\u03B6'], marker = 'o', label = i.key)
    
plt.legend()
plt.grid(True)
plt.ylabel('\u03B6', rotation = 0)
plt.xlabel('Reynolds')

plt.savefig('Diagrama pierderilor liniare de sarcina in functie de zeta (1).png', dpi = 500)
plt.show()

## Diagrama pierderilor locale de sarcina in functie de zeta si Re (1)

In [ ]:
plt.title("Diagrama pierderilor liniare de sarcina in functie de zeta (2)")

for i in masuratori_pierdere_locala_2:
    df = i.df_local
    plt.plot(df['Re'], df['\u03B6'], marker = 'o', label = i.key)
    
plt.legend()
plt.grid(True)
plt.ylabel('\u03B6', rotation = 0)
plt.xlabel('Reynolds')

plt.savefig("Diagrama pierderilor liniare de sarcina in functie de zeta (2).png", dpi = 500)
plt.show()

## Diagrama pierderilor locale de sarcina in functie de debite (1)

In [ ]:
plt.title("Diagrama pierderilor locale de sarcina efectiva in functie de debite (1)")

for i in masuratori_pierdere_locala_1:
    plt.plot(i.debite, i.pierdere_sarcina(), marker = 'o', label = i.key)
    
plt.legend()
plt.grid(True)
plt.ylabel('h [m]', rotation = 0)
#schimba labeluri debite sa fie in litri
plt.xlabel('Q [m^3/s]')

plt.savefig("Diagrama pierderilor locale de sarcina efectiva in functie de debite (1).png", dpi = 500)
plt.show()

## Diagrama pierderilor locale de sarcina in functie de debite (2)

In [ ]:
plt.title("Diagrama pierderilor locale de sarcina efectiva in functie de debite(2)")

for i in masuratori_pierdere_locala_2:
    plt.plot(i.debite, i.pierdere_sarcina(), marker = 'o', label = i.key)
    
plt.legend()
plt.grid(True)
plt.ylabel('h [m]', rotation = 0)
#schimba labeluri debite sa fie in litri
plt.xlabel('Q [m^3/s]')

plt.savefig("Diagrama pierderilor locale de sarcina efectiva in functie de debite (2).png", dpi = 500)
plt.show()

## Diagrama pierderilor locale de sarcina in functie de viteze (1)

In [ ]:
plt.title("Diagrama pierderilor locale de sarcina in functie de viteze (1)")

for i in masuratori_pierdere_locala_1:
    plt.plot(i.viteza_medie(), i.pierdere_sarcina(), marker = 'o', label = i.key)
    
plt.legend()
plt.grid(True)
plt.ylabel('h [m]', rotation = 0)
plt.xlabel('v [m/s]')

plt.savefig("Diagrama pierderilor locale de sarcina in functie de viteze (1).png", dpi = 500)
plt.show()

## Diagrama pierderilor locale de sarcina in functie de viteze (2)

# Export rezultate

In [ ]:
plt.title("Diagrama pierderilor locale de sarcina in functie de viteze (2)")

for i in masuratori_pierdere_locala_2:
    plt.plot(i.viteza_medie(), i.pierdere_sarcina(), marker = 'o', label = i.key)
    
plt.legend()
plt.grid(True)
plt.ylabel('h [m]', rotation = 0)
plt.xlabel('v [m/s]')

plt.savefig("Diagrama pierderilor locale de sarcina in functie de viteze (2).png", dpi = 500)
plt.show()

In [ ]:
masuratori_pierdere_locala = np.append(masuratori_pierdere_locala_1, masuratori_pierdere_locala_2)
export_excel(masuratori_pierdere_liniara, pierdere = "liniara")
export_excel(masuratori_pierdere_locala, pierdere= "locala")